In [1]:
#importing the required packages
import pandas as pd
import os
import numpy as np
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import math

In [2]:
# changing the working directory
os.chdir('D:/Kaggle')

In [3]:
# importing data
df = pd.read_csv('blore_apartment_data.csv')
geodf = pd.read_csv('apartment_data.csv')

In [4]:
df.head()

,names,Price,Area,Unit Type
0,Salarpuria Sattva Cadenza,39 L -41.65 L,755 sq.ft,1 BHK Apartment
1,Salarpuria Sattva Cadenza,55 L -75 L,1175-1275 sq.ft,2 BHK Apartment
2,Salarpuria Sattva Cadenza,70.04 L -73.30 L,1335-1340 sq.ft,2.5 BHK Apartment
3,Salarpuria Sattva Cadenza,65 L -95 L,1365-1595 sq.ft,3 BHK Apartment
4,Purva Skydale,76.25 L -1.75 Cr,1273-1371 sq.ft,2 BHK Apartment


In [5]:
geodf.head()

,names,lat,lon,geometry
0,Purva Skydale bangalore,12.894033,77.662362,POINT (77.66236169999999 12.894033)
1,Salarpuria Sattva Cadenza bangalore,12.889441,77.640221,POINT (77.64022109999999 12.889441)
2,Shriram Summitt bangalore,12.836068,77.667242,POINT (77.6672418 12.8360678)
3,Shriram Luxor bangalore,13.085249,77.654915,POINT (77.65491539999999 13.085249)
4,Ecolife Elements Of Nature bangalore,12.938728,77.731126,POINT (77.73112619999999 12.9387277)


In [6]:
df.shape

(2660, 4)

In [7]:
geodf.shape

(1124, 4)

In [8]:
# Adding the string "banglore" to names column in df for refencing purpose
df['names'] = df['names'] + ' bangalore'

In [9]:
df.head()

,names,Price,Area,Unit Type
0,Salarpuria Sattva Cadenza bangalore,39 L -41.65 L,755 sq.ft,1 BHK Apartment
1,Salarpuria Sattva Cadenza bangalore,55 L -75 L,1175-1275 sq.ft,2 BHK Apartment
2,Salarpuria Sattva Cadenza bangalore,70.04 L -73.30 L,1335-1340 sq.ft,2.5 BHK Apartment
3,Salarpuria Sattva Cadenza bangalore,65 L -95 L,1365-1595 sq.ft,3 BHK Apartment
4,Purva Skydale bangalore,76.25 L -1.75 Cr,1273-1371 sq.ft,2 BHK Apartment


In [10]:
len(geodf['names'].unique()) - geodf.shape[0]

-106

In [11]:
# deleting the duplicate values
geodf = (geodf.drop_duplicates(keep="first"))

In [12]:
len(geodf['names'].unique()) - geodf.shape[0]

0

In [13]:
df.isnull().sum()

names          0
Price        112
Area         160
Unit Type      0
dtype: int64

In [14]:
(df.isnull().sum()[1] + df.isnull().sum()[2])/df.shape[0]

0.10225563909774436

In [15]:
# removing the null values because they are less than 25% of master data
df = df.dropna()

In [16]:
apartment = df.drop_duplicates(subset=['names'], keep='first')

In [17]:
# keeping the unique values in names columns
apartment.shape

(910, 4)

In [18]:
# splitting the "Price" column into "Starting Price" and "Maximum Price" 
apartment[['Starting Price', 'Maximum Price']] = apartment['Price'].str.split('-', expand=True)

C:\Users\User\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [19]:
apartment.shape

(910, 6)

In [20]:
apartment.head()

,names,Price,Area,Unit Type,Starting Price,Maximum Price
0,Salarpuria Sattva Cadenza bangalore,39 L -41.65 L,755 sq.ft,1 BHK Apartment,39 L,41.65 L
4,Purva Skydale bangalore,76.25 L -1.75 Cr,1273-1371 sq.ft,2 BHK Apartment,76.25 L,1.75 Cr
6,Shriram Summitt bangalore,41.76 L -54 L,714 sq.ft,1 BHK Apartment,41.76 L,54 L
10,Shriram Luxor bangalore,56.64 L -80.06 L,1195 sq.ft,2 BHK Apartment,56.64 L,80.06 L
13,Salarpuria Sattva East Crest bangalore,29.84 L -39.47 L,598 sq.ft,1 BHK Apartment,29.84 L,39.47 L


In [21]:
# sorting into ascending order by "names"
apartment = apartment.sort_values("names", ascending=True).reset_index().drop(columns=['index'])

In [22]:
apartment.head()

,names,Price,Area,Unit Type,Starting Price,Maximum Price
0,42 Queens Square bangalore,78 L -1.56 Cr,1691-2170 sq.ft,3 BHK Apartment,78 L,1.56 Cr
1,A Peace Paramount bangalore,41.78 L -44.72 L,1100-1177 sq.ft,2 BHK Apartment,41.78 L,44.72 L
2,AR Splendor Park bangalore,40.11 L -43.85 L,1146-1253 sq.ft,2 BHK Apartment,40.11 L,43.85 L
3,ARS Bhavisha bangalore,51 L -65 L,1000-1260 sq.ft,2 BHK Apartment,51 L,65 L
4,Aakruthi Aura bangalore,36 L -46.85 L,1115-1129 sq.ft,2 BHK Apartment,36 L,46.85 L


In [23]:
apartment.isnull().sum()

names               0
Price               0
Area                0
Unit Type           0
Starting Price      0
Maximum Price     149
dtype: int64

In [24]:
apartment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   names           910 non-null    object
 1   Price           910 non-null    object
 2   Area            910 non-null    object
 3   Unit Type       910 non-null    object
 4   Starting Price  910 non-null    object
 5   Maximum Price   761 non-null    object
dtypes: object(6)
memory usage: 42.8+ KB


In [25]:
# We dont use "Maximum Price" in this analysis, hence filling 0 in Null values of "Maximum Price" 
apartment = apartment.fillna(0)

In [26]:
apartment.isnull().sum()

names             0
Price             0
Area              0
Unit Type         0
Starting Price    0
Maximum Price     0
dtype: int64

In [27]:
# merging 2 datas
apartment_data = pd.merge(left=apartment, right=geodf, how='left')

In [28]:
apartment_data.isnull().sum()

names              0
Price              0
Area               0
Unit Type          0
Starting Price     0
Maximum Price      0
lat               99
lon               99
geometry          99
dtype: int64

In [29]:
(apartment_data.isnull().sum()[6])/apartment_data.shape[0]

0.1087912087912088

In [30]:
apartment_data = apartment_data.dropna()

In [31]:
# Visualizing using folium
m1 = folium.Map(location=[12.972442, 77.580643], tiles='cartodbpositron', zoom_start=13)

In [32]:
m1

In [33]:
mc = MarkerCluster()

for idx, row in apartment_data.iterrows():
    if not math.isnan(row['lon']) and not math.isnan(row['lat']):
        tooltip = tooltip = '<strong>' + row['names'] + '</strong>'
        mc.add_child(Marker([row['lat'], row['lon']], tooltip =tooltip))

In [34]:
m1.add_child(mc)

In [35]:
# when we zoom out we can see 2 apartments which are out of the bangalore city
apartment_data.loc[apartment_data['names'] == 'SD Aashrita bangalore']

,names,Price,Area,Unit Type,Starting Price,Maximum Price,lat,lon,geometry
630,SD Aashrita bangalore,30.15 L -39.49 L,1005-1095 sq.ft,2 BHK Apartment,30.15 L,39.49 L,43.969515,-99.901813,POINT (-99.9018131 43.9695148)


In [36]:
apartment_data.loc[apartment_data['names'] == 'Hiranandani Horizon bangalore']

,names,Price,Area,Unit Type,Starting Price,Maximum Price,lat,lon,geometry
308,Hiranandani Horizon bangalore,72.08 L -85.87 L,1330-1350 sq.ft,2 BHK Apartment,72.08 L,85.87 L,19.11538,72.909144,POINT (72.90914359999999 19.1153798)


In [37]:
# deleting the apartments with wrong data
apartment_data.drop(index=[630, 308], inplace=True)

In [38]:
m2 = folium.Map(location=[12.972442, 77.580643], tiles='cartodbpositron', zoom_start=13)

mc = MarkerCluster()
for idx, row in apartment_data.iterrows():
    if not math.isnan(row['lon']) and not math.isnan(row['lat']):
        tooltip = tooltip = '<strong>' + row['names'] + '</strong>'
        mc.add_child(Marker([row['lat'], row['lon']], tooltip =tooltip))

In [39]:
m2.add_child(mc)

In [41]:
#final check
apartment_data.head()

,names,Price,Area,Unit Type,Starting Price,Maximum Price,lat,lon,geometry
0,42 Queens Square bangalore,78 L -1.56 Cr,1691-2170 sq.ft,3 BHK Apartment,78 L,1.56 Cr,12.852557,77.782533,POINT (77.7825326 12.8525569)
1,A Peace Paramount bangalore,41.78 L -44.72 L,1100-1177 sq.ft,2 BHK Apartment,41.78 L,44.72 L,13.114182,77.566656,POINT (77.56665579999999 13.1141821)
2,AR Splendor Park bangalore,40.11 L -43.85 L,1146-1253 sq.ft,2 BHK Apartment,40.11 L,43.85 L,13.042217,77.655292,POINT (77.6552918 13.042217)
3,ARS Bhavisha bangalore,51 L -65 L,1000-1260 sq.ft,2 BHK Apartment,51 L,65 L,12.859661,77.765624,POINT (77.7656243 12.8596605)
4,Aakruthi Aura bangalore,36 L -46.85 L,1115-1129 sq.ft,2 BHK Apartment,36 L,46.85 L,12.965070,77.751088,POINT (77.75108770000001 12.9650697)


In [42]:
#exporting the clean data
apartment_data.to_csv('apartment_data.csv', index=False)